# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, make_scorer, f1_score, fbeta_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from scipy.stats import hmean
from scipy.stats.mstats import gmean
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response.db')
df = pd.read_sql("SELECT * FROM Disaster_Response", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):

    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    for tok in tokens:

        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(10)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [9]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = pipeline.predict(msg)
print(Y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'buildings' 'direct_report']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_pred = pipeline.predict(X_test)

In [11]:
overall_accuracy = (Y_pred == Y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.35% 



In [12]:
Y_pred_df = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_df[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

          0       0.61      0.45      0.52      1550
          1       0.84      0.91      0.87      4951
          2       1.00      0.15      0.26        53

avg / total       0.78      0.80      0.78      6554

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5415
          1       0.81      0.43      0.56      1139

avg / total       0.88      0.88      0.87      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

------------------------------------------------------

FEATURE: aid_related

    

### 6. Improve your model
Use grid search to find better parameters. 

In [48]:
def multioutput_fscore(Y_true,Y_pred,beta=1):
    score_list = []
    if isinstance(Y_pred, pd.DataFrame) == True:
        Y_pred = Y_pred.values
    if isinstance(Y_true, pd.DataFrame) == True:
        Y_true = Y_true.values
    for column in range(0,Y_true.shape[1]):
        score = fbeta_score(Y_true[:,column],Y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [49]:
#model = new_model_pipeline()

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[5, 10]}


scorer = make_scorer(multioutput_fscore,greater_is_better = True)

cv = GridSearchCV(pipeline, param_grid=parameters, scoring = scorer,verbose = 10)

tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9307770682417511, total=  48.1s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9301486733912714, total=  47.8s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9302040167377527, total=  48.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  8.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.9299324554000817, total=  48.2s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9337248437077929, total=  42.5s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9336949028784426, total=  42.8s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9332950201962095, total=  42.6s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9328089318089142, total= 1.5min
[CV] clf__estimator__min_samples_split=5, clf__

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 64.3min finished


In [50]:
tuned_model.cv_results_

{'mean_fit_time': array([ 39.67278115,  35.37707337,  39.42397507,  34.46058369,
         76.57117184,  66.63644544,  77.30097802,  64.46016232,
         36.67727621,  33.58159097,  35.80002387,  32.59379443,
         70.10156417,  63.49073998,  69.40524999,  60.30547937]),
 'std_fit_time': array([ 0.47056228,  0.14083261,  0.14080448,  0.09722324,  0.33564206,
         0.59309007,  0.22774494,  0.11769126,  0.21051254,  0.09718195,
         0.14930166,  0.16195998,  0.56828684,  0.29047222,  0.34900562,
         0.19943767]),
 'mean_score_time': array([  8.58704265,   8.26714039,   8.57556566,   8.17402895,
         10.97876263,   9.96627975,  10.83410541,   9.92295559,
          8.58331752,   8.15958556,   8.48835937,   8.14424547,
         10.92985765,  10.05448453,  10.84957107,   9.88287028]),
 'std_score_time': array([ 0.04704313,  0.11066596,  0.02868869,  0.05655989,  0.1736548 ,
         0.08373933,  0.04427652,  0.05825392,  0.093336  ,  0.06057397,
         0.06358894,  0.07

In [51]:
np.max(tuned_model.cv_results_['mean_test_score'])

0.93513062976916672

In [52]:
tuned_model.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': False,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [53]:
#evaluating metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

#eval_metrics1 = eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

#print(eval_metrics1)

tuned_pred_test_df = pd.DataFrame(tuned_pred_test, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],tuned_pred_test_df[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

          0       0.70      0.46      0.55      1550
          1       0.84      0.94      0.89      4951
          2       0.52      0.42      0.46        53

avg / total       0.81      0.82      0.81      6554

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5415
          1       0.79      0.52      0.62      1139

avg / total       0.88      0.89      0.88      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

------------------------------------------------------

FEATURE: aid_related

    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [37]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [42]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25]}


scorer = make_scorer(multioutput_fscore,greater_is_better = True)

cv2 = GridSearchCV(pipeline2, param_grid=parameters, scoring = scorer,verbose = 10)

tuned_model2 = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9277887199478341, total=  33.1s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   47.9s remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9304901052264236, total=  33.0s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9286751277840292, total=  33.2s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  6.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.9294319047038674, total=  32.0s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.1min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.928058041979719, total=  30.1s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9303183856442652, total=  30.0s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9294755646737105, total=  30.0s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9311350680801351, total=  50.6s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9334168541043992, total=  50.4s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tf

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 21.9min finished


### 9. Export your model as a pickle file

In [55]:
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.